<a href="https://colab.research.google.com/github/tejasspawar/face-mask-detection/blob/main/Face_Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [8]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import numpy as np
import cv2
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# Creating X and y

## Sample of original image

In [12]:
path = '/content/drive/MyDrive/Face mask data/mask/1.png'
img = cv2.imread(path)
img.shape

(96, 96, 3)

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
plt.imshow(img)

## Appending With mask images

In [ ]:
 X = []
 X_scaled = []
 y = []
 for i in range(1,595):
   try:
     path = '/content/drive/MyDrive/Face mask data/mask/{}.png'.format(i)
     img = cv2.imread(path)    
     X.append(img)
     X_scaled.append(img/255)
     y.append(1)
   except:
     pass


## Appending without mask images

In [ ]:
for i in range(1,617):
   try:
     path = '/content/drive/MyDrive/Face mask data/without_mask/{}.png'.format(i)
     img = cv2.imread(path)    
     X.append(img)
     X_scaled.append(img/255)
     y.append(0)
   except:
     pass  

# Train-Test split

In [ ]:
X = np.array(X)
X_scaled = np.array(X_scaled)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.15, random_state=0, stratify=y)

# Training CNN

In [ ]:
cnn1 = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(96, 96, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [ ]:
cnn1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn1.fit(X_train, y_train, epochs=5)

# Classification report of test data


In [ ]:
from sklearn.metrics import confusion_matrix , classification_report
y_pred = cnn1.predict(X_test)
y_pred_classes = [np.argmax(element) for element in y_pred]

print("Classification Report: \n", classification_report(y_test, y_pred_classes))

# Testing


In [ ]:
test_path_wm = '/content/drive/MyDrive/Face mask data/test/wmask_test1.png'
test_path_m = '/content/drive/MyDrive/Face mask data/test/mask_test1.png'
img_wm = cv2.imread(test_path_wm)
img_wm = cv2.resize(img_wm,(96,96))
img_wm= img_wm/255
img_m = cv2.imread(test_path_m)
img_m = cv2.resize(img_m,(96,96))
img_m= img_m/255

In [ ]:
plt.imshow(img_wm)

In [ ]:
# Not Wearing mask
if cnn1.predict(np.array([img_wm]))[0][1]>0.5:
  print('Person is wearing a mask')
else:
  print('Person is not wearing a mask')


In [ ]:
plt.imshow(img_m)

In [ ]:
# Wearing mask
if cnn1.predict(np.array([img_m]))[0][1]>0.5:
  print('Person is wearing a mask')
else:
  print('Person is not wearing a mask')


# Training CNN with Data Augmentaion


In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(96, 
                                                              96,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2),
  ]
)

In [ ]:
# Previous
plt.imshow(X_scaled[0])

In [ ]:
# After augmentation
plt.imshow(data_augmentation(X_scaled)[0])

In [ ]:
cnn2 = models.Sequential([
    data_augmentation,
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [ ]:
cnn2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn2.fit(X_train, y_train, epochs=5)

# Classification report of test data


In [ ]:
from sklearn.metrics import confusion_matrix , classification_report
y_pred = cnn2.predict(X_test)
y_pred_classes = [np.argmax(element) for element in y_pred]

print("Classification Report: \n", classification_report(y_test, y_pred_classes))

# Testing

In [ ]:
test_path_wm = '/content/drive/MyDrive/Face mask data/test/wmask_test1.png'
test_path_m = '/content/drive/MyDrive/Face mask data/test/mask_test1.png'
img_wm = cv2.imread(test_path_wm)
img_wm = cv2.resize(img_wm,(96,96))
img_wm= img_wm/255
img_m = cv2.imread(test_path_m)
img_m = cv2.resize(img_m,(96,96))
img_m= img_m/255

In [ ]:
plt.imshow(img_wm)

In [ ]:
# Not Wearing mask
if cnn2.predict(np.array([img_wm]))[0][1]>0.5:
  print('Person is wearing a mask')
else:
  print('Person is not wearing a mask')


In [ ]:
plt.imshow(img_m)

In [ ]:
# Wearing mask
if cnn2.predict(np.array([img_m]))[0][1]>0.5:
  print('Person is wearing a mask')
else:
  print('Person is not wearing a mask')


# Training CNN using keras tuner

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(96,96,3)
    ),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(2, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(X_train,y_train,epochs=3,validation_split=0.1)

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_split=0.1, initial_epoch=3)

# Classification report of test data


In [ ]:
from sklearn.metrics import confusion_matrix , classification_report
y_pred = model.predict(X_test)
y_pred_classes = [np.argmax(element) for element in y_pred]

print("Classification Report: \n", classification_report(y_test, y_pred_classes))

# Testing

In [ ]:
test_path_wm = '/content/drive/MyDrive/Face mask data/test/wmask_test1.png'
test_path_m = '/content/drive/MyDrive/Face mask data/test/mask_test1.png'
img_wm = cv2.imread(test_path_wm)
img_wm = cv2.resize(img_wm,(96,96))
img_wm= img_wm/255
img_m = cv2.imread(test_path_m)
img_m = cv2.resize(img_m,(96,96))
img_m= img_m/255

In [ ]:
plt.imshow(img_wm)

In [ ]:
# Not Wearing mask
if model.predict(np.array([img_wm]))[0][1]>0.5:
  print('Person is wearing a mask')
else:
  print('Person is not wearing a mask')


In [ ]:
plt.imshow(img_m)

In [ ]:
# Wearing mask
if model.predict(np.array([img_m]))[0][1]>0.5:
  print('Person is wearing a mask')
else:
  print('Person is not wearing a mask')
